In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pprint
import os
pp = pprint.PrettyPrinter(indent=4)

In [3]:
x_file, y_file = np.load("../data/x.npz"), np.load("../data/y.npz")
X = x_file['x']
Y = y_file['y_compress'].astype(np.int8)
print(f"X's shape is {X.shape}, Y's shape is {Y.shape}")
# _, inv_idx, path_count = np.unique(Y, return_inverse=True, return_counts=True)
# weight = path_count[inv_idx] / path_count.sum()

X's shape is (53022, 376), Y's shape is (53022, 2194)


In [4]:
# define the model
CONV_KERNEL_SZ = 9
CONV_STRIDE_SZ = 3
CONV_ACTVATION = "relu"
CONV_CHANNEL = 3
DROP_OUT_PROB = .25
DIM_X, DIM_Y = X.shape[1], Y.shape[1]

MODEL_SAVE_DIR = "../model/c2d2"
if os.path.exists(MODEL_SAVE_DIR):
    model = tf.keras.models.load_model(MODEL_SAVE_DIR)
else:
    model = keras.Sequential([
        keras.layers.Conv1D(CONV_CHANNEL, CONV_KERNEL_SZ, strides=CONV_STRIDE_SZ, activation=CONV_ACTVATION, input_shape=(DIM_X, 1)),
        keras.layers.Dropout(rate=DROP_OUT_PROB),    
        keras.layers.Conv1D(CONV_CHANNEL, CONV_KERNEL_SZ, strides=CONV_STRIDE_SZ, activation=CONV_ACTVATION),      
        # keras.layers.Dropout(rate=DROP_OUT_PROB),
        # keras.layers.Conv1D(CONV_CHANNEL, CONV_KERNEL_SZ, strides=CONV_STRIDE_SZ, activation=CONV_ACTVATION),
        keras.layers.Flatten(),  
        # keras.layers.Dense(4096),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(DIM_Y, activation='sigmoid')
    ])
# model = keras.Sequential([
#     keras.layers.Dense(DIM_Y, activation='relu', input_dim=DIM_X),
#     keras.layers.Dense(DIM_Y, activation='relu'),
# ])
opt = keras.optimizers.Adam(learning_rate=0.0001)
# better to use the precision
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy', keras.metrics.Recall(thresholds=0.5)])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 123, 3)            30        
_________________________________________________________________
dropout (Dropout)            (None, 123, 3)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 39, 3)             84        
_________________________________________________________________
flatten (Flatten)            (None, 117)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               11800     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2194)              2

In [5]:
X_expand = np.expand_dims(X.astype(np.float32), axis=-1)
# X_expand = np.expand_dims(X_expand.astype(np.float32), axis=-1)
data = tf.data.Dataset.from_tensor_slices((X_expand, Y.astype(np.int32)))

In [6]:
data.shuffle(X.shape[0])
n_train = int(X.shape[0] * 0.75)
train, test = data.take(n_train), data.skip(n_train)   # 75% split

In [7]:
train = train.batch(100)
valid_data = test
valid_data = valid_data.batch(100)
train.element_spec, valid_data.element_spec

((TensorSpec(shape=(None, 376, 1), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 2194), dtype=tf.int32, name=None)),
 (TensorSpec(shape=(None, 376, 1), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 2194), dtype=tf.int32, name=None)))

In [8]:
history = model.fit(
    train,
    # batch_size=200,
    epochs=20,
    validation_data=valid_data
)

Epoch 1/20
398/398 [==============================] - 8s 18ms/step - loss: 0.5613 - binary_accuracy: 0.7178 - recall: 0.6993 - val_loss: 0.3907 - val_binary_accuracy: 0.8377 - val_recall: 0.7628
Epoch 2/20
398/398 [==============================] - 7s 17ms/step - loss: 0.4441 - binary_accuracy: 0.7581 - recall: 0.7105 - val_loss: 0.3897 - val_binary_accuracy: 0.8429 - val_recall: 0.7667
Epoch 3/20
398/398 [==============================] - 7s 17ms/step - loss: 0.4309 - binary_accuracy: 0.7650 - recall: 0.6975 - val_loss: 0.3872 - val_binary_accuracy: 0.8465 - val_recall: 0.7643
Epoch 4/20
398/398 [==============================] - 7s 17ms/step - loss: 0.4255 - binary_accuracy: 0.7702 - recall: 0.6854 - val_loss: 0.3857 - val_binary_accuracy: 0.8492 - val_recall: 0.7612
Epoch 5/20
398/398 [==============================] - 7s 17ms/step - loss: 0.4223 - binary_accuracy: 0.7740 - recall: 0.6758 - val_loss: 0.3846 - val_binary_accuracy: 0.8510 - val_recall: 0.7588
Epoch 6/20
398/398 [=====

In [9]:
model.save(MODEL_SAVE_DIR)

INFO:tensorflow:Assets written to: ../model/c2d2\assets


In [10]:
test = test.batch(100)
eval_results = model.evaluate(test)

133/133 [==============================] - 1s 8ms/step - loss: 0.3791 - binary_accuracy: 0.8575 - recall: 0.7497
